# Start Engines, Get Main Census File

In [47]:
# SQLAlchemy
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Table, Column, Integer, String, Float, MetaData

#pandas for data manipulation and export
import pandas as pd

#________*_________*_________*_________*_________*_________*_________*_________*
db_uri = "sqlite:///db.sqlite"
#engine = create_engine(db_uri, echo = True)
engine = create_engine(db_uri, echo = False)

#________*_________*_________*_________*_________*_________*_________*_________*

census = pd.read_csv("data/S2802.csv", na_values = ["-"])
census = census.astype({"fips_id": "object"})
census["county_name"] = [x.replace(" ", "") for x in census["county_name"]]
census["state"] = [x.replace(" ", "") for x in census["state"]]
dt = census.dtypes[1:10] 
print(f'data types of top 10 columns: {dt}')



data types of top 10 columns: fips_id                                                      object
GeographicAreaName                                           object
tract_name                                                   object
county_name                                                  object
state                                                        object
E_Households                                                  int64
E_Households_Race_Ethnicity_Whitealone                        int64
E_Computer_Broadband_Households                               int64
E_Computer_Broadband_Households_Race_Ethnicity_Whitealone     int64
dtype: object


# Get Census Add-Ons

In [48]:
#________*_________*_________*_________*_________*_________*_________*_________*
#Get LatLon
latlon = pd.read_csv("data/fips_latlon.csv")

#________*_________*_________*_________*_________*_________*_________*_________*
#Get Region
region = pd.read_csv("data/region.csv")
dt = region.dtypes
print(f'region data types are: {dt}')

missingcol = pd.read_csv("data/missingcol.csv")
missingcol

region data types are: County        object
CountyName    object
Region        object
dtype: object


,fips_id,E_Computer_Broadband_Household
0,41001950100,2020
1,41001950200,2764
2,41001950300,1943
3,41001950400,2100
4,41001950500,1751
...,...,...
829,41071030702,6044
830,41071030801,4266
831,41071030802,3199
832,41071030900,4175


In [49]:
pd.merge(census, region, left_on="county_name", right_on="CountyName")

,id,fips_id,GeographicAreaName,tract_name,county_name,state,E_Households,E_Households_Race_Ethnicity_Whitealone,E_Computer_Broadband_Households,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,...,E_Households_EducationBachelorsPlus,E_Computer_Broadband_Households_EducationBachelorsPlus,PE_Computer_Broadband_Households_EducationBachelorsPlus,E_Computer_NoInternet_Households_EducationBachelorsPlus,PE_Computer_NoInternet_Households_EducationBachelorsPlus,E_NoComputer_Households_EducationBachelorsPlus,PE_NoComputer_Households_EducationBachelorsPlus,County,CountyName,Region
0,1400000US41001950100,41001950100,"Census Tract 9501, Baker County, Oregon",Census Tract 9501,Baker,Oregon,2806,2659,2020,1887,...,449,419,93.3,1,0.2,27,6.0,Baker County,Baker,East
1,1400000US41001950200,41001950200,"Census Tract 9502, Baker County, Oregon",Census Tract 9502,Baker,Oregon,3435,3247,2764,2685,...,661,607,91.8,14,2.1,40,6.1,Baker County,Baker,East
2,1400000US41001950300,41001950300,"Census Tract 9503, Baker County, Oregon",Census Tract 9503,Baker,Oregon,2419,2244,1943,1772,...,231,224,97.0,0,0.0,7,3.0,Baker County,Baker,East
3,1400000US41001950400,41001950400,"Census Tract 9504, Baker County, Oregon",Census Tract 9504,Baker,Oregon,2634,2413,2100,1979,...,493,424,86.0,36,7.3,33,6.7,Baker County,Baker,East
4,1400000US41001950500,41001950500,"Census Tract 9505, Baker County, Oregon",Census Tract 9505,Baker,Oregon,2166,1982,1751,1567,...,429,403,93.9,16,3.7,10,2.3,Baker County,Baker,East
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825,1400000US41071030702,41071030702,"Census Tract 307.02, Yamhill County, Oregon",Census Tract 307.02,Yamhill,Oregon,7347,6534,6044,5248,...,1053,936,88.9,69,6.6,48,4.6,Yamhill County,Yamhill,Northwest
826,1400000US41071030801,41071030801,"Census Tract 308.01, Yamhill County, Oregon",Census Tract 308.01,Yamhill,Oregon,5311,4429,4266,3634,...,557,486,87.3,49,8.8,22,3.9,Yamhill County,Yamhill,Northwest
827,1400000US41071030802,41071030802,"Census Tract 308.02, Yamhill County, Oregon",Census Tract 308.02,Yamhill,Oregon,3914,3008,3199,2309,...,291,261,89.7,22,7.6,8,2.7,Yamhill County,Yamhill,Northwest
828,1400000US41071030900,41071030900,"Census Tract 309, Yamhill County, Oregon",Census Tract 309,Yamhill,Oregon,4829,4285,4175,3660,...,731,678,92.7,40,5.5,13,1.8,Yamhill County,Yamhill,Northwest


# Merge Results into Census Main DF

In [50]:
#________*_________*_________*_________*_________*_________*_________*_________*
#Merge missing column, on fips_code
df_col = census.merge(missingcol, how="inner", on="fips_id")

#Merge Region on County
df_reg = df_col.merge(region, how="inner", left_on="county_name", right_on="CountyName")

#Merge Coordinates on fips_code
census_df = df_reg.merge(latlon, how="inner", left_on=["id","fips_id"], right_on=["GEO_ID","StateTractCode"])
census_df.iloc[0,10:50]
#census_df.PE_Computer_Broadband_Household
census_df.E_Computer_Broadband_Household

0      2020
1      2764
2      1943
3      2100
4      1751
       ... 
825    6044
826    4266
827    3199
828    4175
829    3435
Name: E_Computer_Broadband_Household, Length: 830, dtype: int64

# Get FCC Data

In [51]:
#________*_________*_________*_________*_________*_________*_________*_________*
#Get FCC
fcc = pd.read_csv("data/fcc_data.csv") #, na_values = ["-"]
fcc = fcc.astype({"tract": "object"})

#subset columns
fcc_df = fcc[["dbaname","hocofinal","stateabbr","blockcode", "tract", "techcode","consumer","maxaddown","maxadup"]]
fcc_df
dt = fcc_df.dtypes
print(f"fcc dtypes: {dt}")


fcc dtypes: dbaname       object
hocofinal     object
stateabbr     object
blockcode    float64
tract         object
techcode       int64
consumer       int64
maxaddown      int64
maxadup        int64
dtype: object


# Send to SQLite

In [52]:
#________*_________*_________*_________*_________*_________*_________*_________*
# To SQL census
census_df.to_sql("census", con=engine, if_exists="replace", index_label=None)

#To SQL fcc
fcc_df.to_sql("fcc", con=engine, if_exists="replace", index_label=None)

#Test
print(engine.execute("SELECT * FROM census LIMIT 2").fetchall())
print(engine.execute("SELECT * FROM fcc LIMIT 2").fetchall())

[(0, '1400000US41001950100', 41001950100, 'Census Tract 9501, Baker County, Oregon', 'Census Tract 9501', 'Baker', 'Oregon', 2806, 2659, 2020, 1887, 72.0, 71.0, 555, 541, 19.8, 20.3, 199, 199, 7.1, 7.5, 26, 26, 100.0, 0, 0.0, 0, 0.0, 24, 24, 100.0, 0, 0.0, 0, 0.0, 22, 22, 100.0, 0, 0.0, 0, 0.0, 9, 2, 22.2, 7, 77.8, 0, 0.0, 47, 47, 100.0, 0, 0.0, 0, 0.0, 19, 12, 63.2, 7, 36.8, 0, 0.0, 161, 79, 49.1, 82, 50.9, 0, 0.0, 2535, 1843, 72.7, 461, 18.2, 199, 7.9, 2217, 1614, 72.8, 372, 16.8, 199, 9.0, 249, 103, 41.4, 96, 38.6, 50, 20.1, 1519, 1092, 71.9, 275, 18.1, 122, 8.0, 449, 419, 93.3, 1, 0.2, 27, 6.0, 2020, 'Baker County', 'Baker', 'East', '1400000US41001950100', 41001950100, 44.615008200000005, -118.15274140000001), (1, '1400000US41001950200', 41001950200, 'Census Tract 9502, Baker County, Oregon', 'Census Tract 9502', 'Baker', 'Oregon', 3435, 3247, 2764, 2685, 80.5, 82.7, 255, 238, 7.4, 7.3, 416, 324, 12.1, 10.0, 25, 21, 84.0, 4, 16.0, 0, 0.0, 41, 41, 100.0, 0, 0.0, 0, 0.0, 0, 0, None, 

In [53]:
query = "SELECT Latitude, Longitude, Region, County, fips_id, PE_Computer_Broadband_Household, E_Computer_Broadband_Household FROM census"
results = engine.execute(query).fetchall()
print(results)

result = engine.execute(query)
#print(result)

[(44.615008200000005, -118.15274140000001, 'East', 'Baker County', 41001950100, 72.0, 2020), (44.778647, -117.87515259999998, 'East', 'Baker County', 41001950200, 80.5, 2764), (44.5605166, -117.57923570000001, 'East', 'Baker County', 41001950300, 80.3, 1943), (44.79115, -117.8172115, 'East', 'Baker County', 41001950400, 79.7, 2100), (44.9038966, -117.78576629999999, 'East', 'Baker County', 41001950500, 80.8, 1751), (44.851074, -117.2473647, 'East', 'Baker County', 41001950600, 78.0, 1521), (44.535963, -123.2632065, 'Northwest', 'Benton County', 41003000100, 94.9, 7593), (44.5379586, -123.3077291, 'Northwest', 'Benton County', 41003000202, 95.0, 4849), (44.599641600000005, -123.2820909, 'Northwest', 'Benton County', 41003000400, 95.8, 7545), (44.632301399999996, -123.2601892, 'Northwest', 'Benton County', 41003000500, 90.0, 2958), (44.610372, -123.1979735, 'Northwest', 'Benton County', 41003000600, 84.4, 3700), (44.5849205, -123.3117656, 'Northwest', 'Benton County', 41003000900, 96.0, 

In [54]:
results = engine.execute(query)
for result in results:
    print(result[1])

-118.15274140000001
-117.87515259999998
-117.57923570000001
-117.8172115
-117.78576629999999
-117.2473647
-123.2632065
-123.3077291
-123.2820909
-123.2601892
-123.1979735
-123.3117656
-123.2584733
-123.27206399999999
-123.27899509999999
-123.26768229999999
-123.1358511
-123.4048254
-123.5302807
-123.3334335
-123.26695749999999
-123.2930023
-123.3596186
-123.36057890000001
-122.67116159999999
-122.68855149999999
-122.7334616
-122.71528070000001
-122.73714199999999
-122.72922209999999
-122.7048436
-122.69148090000002
-122.6676637
-122.66742820000002
-122.6581504
-122.63928829999999
-122.6257861
-122.6546685
-122.64009309999999
-122.6277149
-122.6109135
-122.6103187
-122.6425893
-122.64412730000001
-122.6230258
-122.6071077
-122.58554380000001
-122.58603459999999
-122.6213643
-122.60374809999999
-122.61393729999999
-122.60242609999999
-122.5897297
-122.57902390000001
-122.51264679999998
-122.5422521
-122.5605543
-122.5515456
-122.5746003
-122.54869850000001
-122.55518130000002
-122.523881

In [89]:
race_query = "SELECT Region,E_Households,E_Households_Race_Ethnicity_Whitealone,E_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Households_Race_Ethnicity_Asianalone,E_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Households_Race_Ethnicity_Someotherracealone,E_Households_Race_Ethnicity_Twoormoreraces,E_Computer_Broadband_Households,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,E_Computer_Broadband_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Computer_Broadband_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Computer_Broadband_Households_Race_Ethnicity_Asianalone,E_Computer_Broadband_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Computer_Broadband_Households_Race_Ethnicity_Someotherracealone,E_Computer_Broadband_Households_Race_Ethnicity_Twoormoreraces,E_Computer_NoInternet_Households,E_Computer_NoInternet_Households_Race_Ethnicity_Whitealone,E_Computer_NoInternet_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Computer_NoInternet_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Computer_NoInternet_Households_Race_Ethnicity_Asianalone,E_Computer_NoInternet_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Computer_NoInternet_Households_Race_Ethnicity_Someotherracealone,E_Computer_NoInternet_Households_Race_Ethnicity_Twoormoreraces,E_NoComputer_Households_Race_Ethnicity_Whitealone,E_NoComputer_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_NoComputer_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_NoComputer_Households_Race_Ethnicity_Asianalone,E_NoComputer_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_NoComputer_Households_Race_Ethnicity_Someotherracealone,E_NoComputer_Households_Race_Ethnicity_Twoormoreraces from census"
race_columns ="Region,E_Households,E_Households_Race_Ethnicity_Whitealone,E_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Households_Race_Ethnicity_Asianalone,E_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Households_Race_Ethnicity_Someotherracealone,E_Households_Race_Ethnicity_Twoormoreraces,E_Computer_Broadband_Households,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,E_Computer_Broadband_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Computer_Broadband_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Computer_Broadband_Households_Race_Ethnicity_Asianalone,E_Computer_Broadband_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Computer_Broadband_Households_Race_Ethnicity_Someotherracealone,E_Computer_Broadband_Households_Race_Ethnicity_Twoormoreraces,E_Computer_NoInternet_Households,E_Computer_NoInternet_Households_Race_Ethnicity_Whitealone,E_Computer_NoInternet_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Computer_NoInternet_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Computer_NoInternet_Households_Race_Ethnicity_Asianalone,E_Computer_NoInternet_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Computer_NoInternet_Households_Race_Ethnicity_Someotherracealone,E_Computer_NoInternet_Households_Race_Ethnicity_Twoormoreraces,E_NoComputer_Households_Race_Ethnicity_Whitealone,E_NoComputer_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_NoComputer_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_NoComputer_Households_Race_Ethnicity_Asianalone,E_NoComputer_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_NoComputer_Households_Race_Ethnicity_Someotherracealone,E_NoComputer_Households_Race_Ethnicity_Twoormoreraces from census".split(",")

In [100]:
tooth = engine.execute(race_query).fetchall()
data_frame = pd.DataFrame(tooth)
data_frame.columns = race_columns
data_frame.groupby("Region").sum()

,E_Households,E_Households_Race_Ethnicity_Whitealone,E_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_Households_Race_Ethnicity_Asianalone,E_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Households_Race_Ethnicity_Someotherracealone,E_Households_Race_Ethnicity_Twoormoreraces,E_Computer_Broadband_Households,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,...,E_Computer_NoInternet_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_Computer_NoInternet_Households_Race_Ethnicity_Someotherracealone,E_Computer_NoInternet_Households_Race_Ethnicity_Twoormoreraces,E_NoComputer_Households_Race_Ethnicity_Whitealone,E_NoComputer_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone,E_NoComputer_Households_Race_Ethnicity_BlackorAfricanAmericanalone,E_NoComputer_Households_Race_Ethnicity_Asianalone,E_NoComputer_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone,E_NoComputer_Households_Race_Ethnicity_Someotherracealone,E_NoComputer_Households_Race_Ethnicity_Twoormoreraces from census
Region,,,,,,,,,,,,,,,,,,,,,
East,424846,384187,9301,2097,4049,935,11252,13025,355586,322719,...,83,1251,908,25149,1439,63,160,52,1117,394
Northwest,2988624,2467416,26528,68242,160126,13555,103403,149354,2660698,2202322,...,2213,10667,10001,108601,1518,5335,4134,794,5796,3950
Southwest,558368,508412,9220,3012,6201,1027,8030,22466,468173,426822,...,22,1582,1545,37192,1111,171,587,59,350,1459


In [ ]:
# E_Computer_Broadband_Households
[
    ["Race", None, 0,],
    ["White","Race",0],
    ["American Indian or Alaska Native", "Race", 0],
    ["Black or African American", "Race", 0],
    ["Asian", "Race", 0],
    ["Native Hawaiian/Other Pacific Islander", "Race", 0],
    ["Other", "Race", 0],
    ["Two or more Races", "Race", 0]
]

# Map

mapquery = "select Latitude, Longitude, Region, county_name, fips_id, PE_Computer_Broadband_Household, E_Computer_Broadband_Household FROM census"

# Treemap

treequeryrace = "SELECT Region 
                    ,E_Households
                    ,E_Households_Race_Ethnicity_Whitealone
                    ,E_Households_Race_Ethnicity_BlackorAfricanAmericanalone
                    ,E_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone
                    ,E_Households_Race_Ethnicity_Asianalone
                    ,E_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone
                    ,E_Households_Race_Ethnicity_Someotherracealone
                    ,E_Households_Race_Ethnicity_Twoormoreraces
                    ,E_Households_Race_Ethnicity_Someotherracealone

                    ,E_Computer_Broadband_Households
    ,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone
    ,E_Computer_Broadband_Households_Race_Ethnicity_BlackorAfricanAmericanalone
    ,E_Computer_Broadband_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone
                    ,E_Computer_Broadband_Households_Race_Ethnicity_Asianalone
               ,E_Computer_Broadband_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone
                    ,E_Computer_Broadband_Households_Race_Ethnicity_Someotherracealone
                    ,E_Computer_Broadband_Households_Race_Ethnicity_Twoormoreraces
                    ,E_Computer_Broadband_Households_Race_Ethnicity_Someotherracealone

    ,E_Computer_NoInternet_Households                    
    ,E_Computer_NoInternet_Households_Race_Ethnicity_Whitealone
    ,E_Computer_NoInternet_Households_Race_Ethnicity_BlackorAfricanAmericanalone
    ,E_Computer_NoInternet_Households_Race_Ethnicity_AmericanIndianandAlaskaNativealone
    ,E_Computer_NoInternet_Households_Race_Ethnicity_Asianalone
   ,E_Computer_NoInternet_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone
,E_Computer_NoInternet_Households_Race_Ethnicity_Someotherracealone
                    ,E_Computer_NoInternet_Households_Race_Ethnicity_Twoormoreraces
 
                    ,E_Households_Race_Ethnicity_HispanicorLatinoorigin(ofanyrace)
                    ,E_Households_Race_Ethnicity_Whitealone,notHispanicorLatino
 ,E_Computer_Broadband_Households_Race_Ethnicity_HispanicorLatinoorigin(ofanyrace)
 ,E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,notHispanicorLatino                   E_Computer_NoInternet_Households_Race_Ethnicity_HispanicorLatinoorigin(ofanyrace)
                    'E_NoComputer_Households_Race_Ethnicity_BlackorAfricanAmericanalone',

                    E_NoComputer_Households_Race_Ethnicity_Someotherracealone
E_NoComputer_Households_Race_Ethnicity_Asianalone
E_NoComputer_Households_Race_Ethnicity_NativeHawaiianandOtherPacificIslanderalone
E_NoComputer_Households_Race_Ethnicity_Someotherracealone
E_NoComputer_Households_Race_Ethnicity_Twoormoreraces
E_NoComputer_Households_Race_Ethnicity_HispanicorLatinoorigin(ofanyrace)
E_NoComputer_Households_Race_Ethnicity_Whitealone,notHispanicorLatino


 from census"


In [76]:
census_df.columns[70:80]

Index(['E_Computer_Broadband_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'PE_Computer_Broadband_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'E_Computer_NoInternet_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'PE_Computer_NoInternet_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'E_NoComputer_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'PE_NoComputer_Households_Race_Ethnicity_Whitealone,notHispanicorLatino',
       'E_Households_Education', 'E_Computer_Broadband_Households_Education',
       'PE_Computer_Broadband_Households_Education',
       'E_Computer_NoInternet_Households_Education'],
      dtype='object')